In [1]:
import pandas as pd
import numpy as np
import datetime
import os

price_threshold=10

In [2]:
last_saturday=datetime.date(2019,1,12) # To be changed to the running Tuesday
def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)
            
most_recent_daily_data=list(recursive_file_gen("/home/jian/BigLots/"))
most_recent_daily_data=[x for x in most_recent_daily_data if ("MediaStormDailySales" in x) and (str(last_saturday) in x)]

if len(most_recent_daily_data)==1:
    most_recent_daily_data=most_recent_daily_data[0]
else:
    most_recent_daily_data=np.nan
    print("Last Weekly Daily Data Error", str(datetime.datetime.now()))

In [3]:
most_recent_daily_data

'/home/jian/BigLots/MediaStorm_2019-01-12/MediaStormDailySales20190115-113411-782.txt'

In [17]:
data=pd.read_table(most_recent_daily_data,dtype=str,sep="|")
print("len_sub_class_id:",data['subclass_id'].apply(lambda x: len(x)).unique())
print("len_class_code_id:",data['class_code_id'].apply(lambda x: len(x)).unique())
data['subclass_id']=data['subclass_id'].apply(lambda x: x.zfill(3))
data['product_comb']=data['class_code_id']+"-"+data['subclass_id']
del data['class_code_id']
del data['subclass_id']
data=data[~pd.isnull(data['customer_id_hashed'])]
data['subclass_transaction_amt']=data['subclass_transaction_amt'].astype(float)
data['subclass_transaction_units']=data['subclass_transaction_units'].astype(int)
data['price']=data['subclass_transaction_amt']/data['subclass_transaction_units']
print("Row_RawData:",data.shape)
print("Unique_id:", len(data['customer_id_hashed'].unique()))
data=data[(data['subclass_transaction_amt']>0) & (data['subclass_transaction_units']>0)]

len_sub_class_id: [1 2 3]
len_class_code_id: [5]
Row_RawData: (6159646, 8)
Unique_id: 1091276


In [18]:
len(data['product_comb'].unique())

1526

In [6]:
taxonomy=pd.read_csv("/home/jian/BigLots/static_files/ProductTaxonomy/MediaStormProductTaxonomy20190101-135843-066.txt",dtype=str,sep="|")
taxonomy['subclass_id']=taxonomy['subclass_id'].apply(lambda x: x.zfill(3))
division_id_id_name=pd.read_table("/home/jian/BigLots/static_files/MediaStorm Data Extract - Division Names.txt",dtype=str,sep="|")
department_id_name=pd.read_table("/home/jian/BigLots/static_files/MediaStorm Data Extract - Department Names.txt",dtype=str,sep="|")
class_id_name=pd.read_table("/home/jian/BigLots/static_files/MediaStorm Data Extract - Class Names.txt",dtype=str,sep="|",encoding ='ISO-8859-1')

In [7]:
data_item_avg_price=data[['product_comb','price']].groupby(['product_comb'])['price'].mean().to_frame().reset_index()
data_item_avg_price=data_item_avg_price.rename(columns={"price":"avg_price"})

data_item_avg_price['class_code_id']=data_item_avg_price['product_comb'].apply(lambda x: x.split("-")[0])
data_item_avg_price['subclass_id']=data_item_avg_price['product_comb'].apply(lambda x: x.split("-")[1])

data_item_avg_price=pd.merge(data_item_avg_price,taxonomy,on=['class_code_id','subclass_id'],how="left")


data_item_avg_price=pd.merge(data_item_avg_price,division_id_id_name,on="division_id",how="left")
data_item_avg_price=pd.merge(data_item_avg_price,department_id_name,on="department_id",how="left")
data_item_avg_price=pd.merge(data_item_avg_price,class_id_name,on="class_code_id",how="left")
data_item_avg_price=data_item_avg_price[['product_comb','avg_price','division_id','division_desc','department_id','department_desc',
                                         'class_code_id','class_code_desc','subclass_id','subclass_desc']]

data_item_avg_price.to_csv("/home/jian/Projects/Big_Lots/Analysis/2018_Q4/Product_Basket/Price_"+str(last_saturday)+".csv",index=False)

In [8]:
# $10 of all items as in the email on 2019-01-14

product_comb_under_10_set=set(data_item_avg_price[data_item_avg_price['avg_price']<price_threshold]['product_comb'].unique().tolist())

print(data.shape)
data=data[~data['product_comb'].isin(product_comb_under_10_set)]
data_under_10=data[data['product_comb'].isin(product_comb_under_10_set)]
data=data.reset_index()
del data['index']
print(data.shape)
dict_item_avg_price=data_item_avg_price.set_index(['product_comb'])['avg_price'].to_dict()

(6082846, 8)
(480861, 8)


In [9]:
product_comb_10_and_above_list=data_item_avg_price[data_item_avg_price['avg_price']>=10]['product_comb'].unique().tolist()

data_transactions_list=data.groupby(['location_id','transaction_dt','transaction_id','customer_id_hashed'])['product_comb'].apply(list).to_frame().reset_index().rename(columns={"product_comb":"basket_list"})
data_transactions_units_sales=data.groupby(['location_id','transaction_dt','transaction_id','customer_id_hashed'])['subclass_transaction_units','subclass_transaction_amt'].sum().reset_index().rename(columns={"subclass_transaction_units":"total_item_units","subclass_transaction_amt":"total_item_revenue"})

data_transactions=pd.merge(data_transactions_list,data_transactions_units_sales,on=['location_id','transaction_dt','transaction_id','customer_id_hashed'],how="left")
data_transactions['basket_str']=data_transactions['basket_list'].apply(lambda x: sorted(x)).astype(str)
data_transactions['transactin_id_given']=[x for x in range(1,len(data_transactions)+1)]
data_transactions['types']=data_transactions['basket_list'].apply(lambda x: len(x))

# To save


data=pd.merge(data,data_transactions,on=["location_id","transaction_dt","transaction_id","customer_id_hashed"],how="left")
apply_func={"subclass_transaction_units":"sum","transactin_id_given":"count"}

single_prod_df=data.groupby(['product_comb'])['subclass_transaction_units','transactin_id_given'].agg(apply_func).reset_index().rename(columns={"subclass_transaction_units":"Total_Units","transactin_id_given":"Total_Trans"})
total_unit=single_prod_df['Total_Units'].sum()
total_trans=single_prod_df['Total_Trans'].sum()

single_prod_df['prob_unit']=single_prod_df['Total_Units']/total_unit
single_prod_df['prob_tran']=single_prod_df['Total_Trans']/total_trans

dict_single_prod_prob_unit=single_prod_df.set_index(['product_comb'])['prob_unit'].to_dict()
dict_single_prod_prob_tran=single_prod_df.set_index(['product_comb'])['prob_tran'].to_dict()


In [10]:
data_basket=data_transactions.groupby(['basket_str'])['total_item_units','total_item_revenue','transactin_id_given'].agg(
            {"total_item_units":"sum","total_item_revenue":"sum","transactin_id_given":"count"}).reset_index().rename(columns={"transactin_id_given":"trans_count"})
data_basket['basket_list']=data_basket['basket_str'].apply(eval)
data_basket['item_types']=data_basket['basket_list'].apply(len)
data_basket=data_basket.sort_values(['item_types','basket_str'])

data_basket=data_basket.reset_index()
del data_basket['index']

unique_id_by_basket=data_transactions.groupby(['basket_str'])['customer_id_hashed'].apply(lambda x: len(set(x))).to_frame().reset_index().rename(columns={'customer_id_hashed':"unique_ids"})
data_basket=pd.merge(data_basket,unique_id_by_basket,on="basket_str",how="left")

In [11]:
# Looks like not needed in this way
'''

from itertools import combinations
def findsubsets(total_set,item_counts):
    return list(set(combinations(total_set, item_counts)))

set_2_comb=sorted(set_2_comb)
set_3_comb=sorted(set_3_comb)
set_4_comb=sorted(set_4_comb)
set_5_comb=sorted(set_5_comb)

i_counter=0
all_trans_list=data_transactions['basket_str'].tolist()
dict_basket_2={}
for basket_2 in set_2_comb:
    basket_item_1=eval(basket)[0]
    basket_item_2=eval(basket)[1]
    
    trans=[x for x in all_trans_list if (basket_item_1 in x) & (basket_item_2 in x)]
    trans_basket_2=len(trans)
    dict_basket_2.update({basket_2:trans_basket_2})
    i_counter+=1
    if i_counter%1000==20:
        print(datetime.datetime.now(),i_counter)
    
'''

"\n\nfrom itertools import combinations\ndef findsubsets(total_set,item_counts):\n    return list(set(combinations(total_set, item_counts)))\n\nset_2_comb=sorted(set_2_comb)\nset_3_comb=sorted(set_3_comb)\nset_4_comb=sorted(set_4_comb)\nset_5_comb=sorted(set_5_comb)\n\ni_counter=0\nall_trans_list=data_transactions['basket_str'].tolist()\ndict_basket_2={}\nfor basket_2 in set_2_comb:\n    basket_item_1=eval(basket)[0]\n    basket_item_2=eval(basket)[1]\n    \n    trans=[x for x in all_trans_list if (basket_item_1 in x) & (basket_item_2 in x)]\n    trans_basket_2=len(trans)\n    dict_basket_2.update({basket_2:trans_basket_2})\n    i_counter+=1\n    if i_counter%1000==20:\n        print(datetime.datetime.now(),i_counter)\n    \n"

In [12]:
output=pd.DataFrame()
for index, row in data_basket.iterrows():
    basket=row['basket_list']
    if len(basket)==1:
        df=pd.DataFrame(row).T
        df['BAI_units']=100
        df['BAI_trans']=100
        price_list=[{basket[0]:np.round(dict_item_avg_price[basket[0]],4)}]
        df['price_list']=[price_list]
        
    elif len(basket)==2:
        item_1=basket[0]
        item_2=basket[1]
        price_list=[{item_1:np.round(dict_item_avg_price[item_1],4)},{item_2:np.round(dict_item_avg_price[item_2],4)}]

        BAI_units=(row['total_item_units']/total_unit) / (dict_single_prod_prob_unit[item_1]*dict_single_prod_prob_unit[item_2]) *100
        BAI_trans=(row['trans_count']/total_trans) / (dict_single_prod_prob_tran[item_1]*dict_single_prod_prob_tran[item_2]) *100
        
        df=pd.DataFrame(row).T
        df['BAI_units']=BAI_units
        df['BAI_trans']=BAI_trans
        df['price_list']=[price_list]
        
    elif len(basket)==3:
        item_1=basket[0]
        item_2=basket[1]
        item_3=basket[2]
        price_list=[{item_1:np.round(dict_item_avg_price[item_1],4)},
                    {item_2:np.round(dict_item_avg_price[item_2],4)},
                    {item_3:np.round(dict_item_avg_price[item_3],4)}]

        BAI_units=(row['total_item_units']/total_unit) / (dict_single_prod_prob_unit[item_1]*dict_single_prod_prob_unit[item_2]*dict_single_prod_prob_tran[item_3]) *100
        BAI_trans=(row['trans_count']/total_trans) / (dict_single_prod_prob_tran[item_1]*dict_single_prod_prob_tran[item_2]*dict_single_prod_prob_tran[item_3]) *100
        
        df=pd.DataFrame(row).T
        df['BAI_units']=BAI_units
        df['BAI_trans']=BAI_trans
        df['price_list']=[price_list]
        
    elif len(basket)==4:
        item_1=basket[0]
        item_2=basket[1]
        item_3=basket[2]
        item_4=basket[3]
        price_list=[{item_1:np.round(dict_item_avg_price[item_1],4)},
                    {item_2:np.round(dict_item_avg_price[item_2],4)},
                    {item_3:np.round(dict_item_avg_price[item_3],4)},
                    {item_4:np.round(dict_item_avg_price[item_4],4)}]
        BAI_units=(row['total_item_units']/total_unit) / (dict_single_prod_prob_unit[item_1]*dict_single_prod_prob_unit[item_2]*dict_single_prod_prob_tran[item_3]*dict_single_prod_prob_tran[item_4]) *100
        BAI_trans=(row['trans_count']/total_trans) / (dict_single_prod_prob_tran[item_1]*dict_single_prod_prob_tran[item_2]*dict_single_prod_prob_tran[item_3]*dict_single_prod_prob_tran[item_4]) *100
        
        df=pd.DataFrame(row).T
        df['BAI_units']=BAI_units
        df['BAI_trans']=BAI_trans
        df['price_list']=[price_list]
    
    elif len(basket)==5:
        item_1=basket[0]
        item_2=basket[1]
        item_3=basket[2]
        item_4=basket[3]
        item_5=basket[4]
        
        BAI_units=(row['total_item_units']/total_unit) / (dict_single_prod_prob_unit[item_1]*dict_single_prod_prob_unit[item_2]*dict_single_prod_prob_tran[item_3]*dict_single_prod_prob_tran[item_4]*dict_single_prod_prob_tran[item_5]) *100
        BAI_trans=(row['trans_count']/total_trans) / (dict_single_prod_prob_tran[item_1]*dict_single_prod_prob_tran[item_2]*dict_single_prod_prob_tran[item_3]*dict_single_prod_prob_tran[item_4]*dict_single_prod_prob_tran[item_5]) *100
        price_list=[{item_1:np.round(dict_item_avg_price[item_1],4)},
                    {item_2:np.round(dict_item_avg_price[item_2],4)},
                    {item_3:np.round(dict_item_avg_price[item_3],4)},
                    {item_4:np.round(dict_item_avg_price[item_4],4)},
                    {item_5:np.round(dict_item_avg_price[item_5],4)}]
        df=pd.DataFrame(row).T
        df['BAI_units']=BAI_units
        df['BAI_trans']=BAI_trans
        df['price_list']=[price_list]
        
    else:
        df=pd.DataFrame(row).T
        

    output=output.append(df)        

In [13]:
output=output.sort_values(['item_types','BAI_trans','BAI_units'],ascending=[True,False,False]).reset_index()
del output['index']

In [14]:
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2018_Q4/Product_Basket'

In [15]:
len("by_transaction_10_plus_only")

27

In [16]:
writer=pd.ExcelWriter('/home/jian/Projects/Big_Lots/Analysis/2018_Q4/Product_Basket/BL_DBasket_Version1_JL_'+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
output=output[['basket_str','basket_list','BAI_trans','BAI_units','item_types','total_item_revenue','total_item_units','trans_count','unique_ids','price_list']]
output.to_excel(writer,"BAI_output_1_to_5",index=False)
data_transactions.to_excel(writer,"by_transaction_10_plus_only",index=False)
single_prod_df=pd.merge(single_prod_df,data_item_avg_price,on="product_comb",how="left")
single_prod_df.to_excel(writer,"single_probilities_10_plus_only",index=False)
data_item_avg_price.to_excel(writer,"item_sub_class_avg_price",index=False)
writer.save()